## Forward kinematics function

In [1]:
import numpy as np
from tqdm import tqdm
import math
# Link lengths (from table)
L1, L2, L3, L4, L5, L6, L7, L8, L9 = 0.055, 0.315, 0.045, 0.108, 0.005, 0.034, 0.015, 0.088, 0.204

def forward_kinematics(theta1, theta2, theta3):
    # Convert theta to radians if necessary
    theta1, theta2, theta3 = np.radians([theta1, theta2, theta3])
    
    # Transformation matrix from frame 0 to frame 1 (shoulder)
    T1 = np.array([
        [np.cos(theta1), -np.sin(theta1), 0, L6],
        [np.sin(theta1), np.cos(theta1), 0, -L4],
        [0, 0, 1, L2 + L3],
        [0, 0, 0, 1]
    ]) @ np.array([
        [1, 0, 0, 0],
        [0, 0, -1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1]
    ])
    
    # Transformation matrix from frame 1 to frame 2 (elbow)
    T2 = np.array([
        [np.cos(theta2), -np.sin(theta2), 0, L7],
        [np.sin(theta2), np.cos(theta2), 0, -L8],
        [0, 0, 1, -L5],
        [0, 0, 0, 1]
    ])
    
    # Transformation matrix from frame 2 to frame 3 (hand)
    T3 = np.array([
        [np.cos(theta3), -np.sin(theta3), 0, 0],
        [np.sin(theta3), np.cos(theta3), 0, -L9],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    
    # Final transformation matrix from frame 0 to the end-effector
    T_final = T1 @ T2 @ T3
    
    # End-effector position (x_0, y_0, z_0) is the last column of the T_final matrix
    P_end_effector = T_final[:3, 3]  # Extract the position
    
    return P_end_effector



## Evolutionary functions 

### 1. Fitness, 
### 2. Tournament selection,
### 3. Two point crossover,
### 4. Mutation

In [2]:
def fitness_function(x_target, y_target, z_target, theta1, theta2, theta3):
    # Calculate predicted end-effector position from forward kinematics
    P_predicted = forward_kinematics(theta1, theta2, theta3)
    P_target = np.array([x_target, y_target, z_target])
    
    # Compute the Euclidean distance (error)
    error = np.linalg.norm(P_target - P_predicted)
    
    return error


def tournament_selection(population, fitness_scores, tournament_size, tournament_prob):
    selected = []
    
    for _ in range(len(population)):
        # Sample 'tournament_size' individuals randomly
        tournament = random.sample(list(zip(population, fitness_scores)), tournament_size)
        
        # Sort the tournament participants by fitness (lower fitness is better)
        tournament = sorted(tournament, key=lambda t: t[1])
        
        # Use the tournament_prob to select the winner
        for i in range(tournament_size):
            if random.random() < tournament_prob:
                selected.append(tournament[i][0])  # Select the ith individual with probability tournament_prob
                break
        # If no one was selected due to probabilities (unlikely), select the best by default
        if len(selected) < _ + 1:
            selected.append(tournament[0][0])  # Add the best individual by default if no one was selected
    
    return selected


import random

def two_point_crossover(individual1, individual2, maxChromosomeLength):
    """ Perform two-point crossover between two chromosomes, limiting the result to maxChromosomeLength. """
    
    # Get the lengths of the parent chromosomes (individuals)
    numInst1 = len(individual1.instructions)
    numInst2 = len(individual2.instructions)
    
    # Select two random crossover points for individual1
    cross1 =  random.randint(0, numInst1)  # Step in blocks of 4
    cross2 =  random.randint(0, numInst1)
    crossOverPoint11 = min(cross1, cross2)
    crossOverPoint12 = max(cross1, cross2)

    # Select two random crossover points for individual2
    cross1 = random.randint(0, numInst2)
    cross2 = random.randint(0, numInst2)
    crossOverPoint21 = min(cross1, cross2)
    crossOverPoint22 = max(cross1, cross2)

    # Create new individuals based on the selected crossover points
    newIndividual1_instructions = (
        individual1.instructions[:crossOverPoint11] + 
        individual2.instructions[crossOverPoint21:crossOverPoint22] + 
        individual1.instructions[crossOverPoint12:]
    )
    
    newIndividual2_instructions = (
        individual2.instructions[:crossOverPoint21] + 
        individual1.instructions[crossOverPoint11:crossOverPoint12] + 
        individual2.instructions[crossOverPoint22:]
    )

    # Limit the length of the new individuals to the maxChromosomeLength
    maxInsLength = math.floor(maxChromosomeLength/4)
    if len(newIndividual1_instructions) > maxInsLength:
        newIndividual1_instructions = newIndividual1_instructions[:maxInsLength]
    if len(newIndividual2_instructions) > maxInsLength:
        newIndividual2_instructions = newIndividual2_instructions[:maxInsLength]

    # Create new Chromosome objects for the offspring
    newIndividual1 = Chromosome(
        num_registers=len(individual1.registers), 
        num_constants=len(individual1.constants)
    )
    newIndividual1.instructions = newIndividual1_instructions
    
    newIndividual2 = Chromosome(
        num_registers=len(individual2.registers), 
        num_constants=len(individual2.constants)
    )
    newIndividual2.instructions = newIndividual2_instructions
    
    return newIndividual1, newIndividual2


def mutation(chromosome, mutation_rate, operators):
    for i in range(len(chromosome.instructions)):
        if random.random() < mutation_rate:
            operator = random.choice(operators)
            dest_reg = random.randint(0, len(chromosome.registers) - 1)
            operand1 = random.choice(chromosome.registers + chromosome.constants)
            operand2 = random.choice(chromosome.registers + chromosome.constants)
            chromosome.instructions[i] = (operator, dest_reg, operand1, operand2)

## Chromosome class

In [3]:
import random

# Define the set of available operators
#OPERATORS = ['+', '-', '*', '/']  # You can extend this list

# Chromosome representation (list of instructions)
class Chromosome:
    def __init__(self, num_registers, num_constants, min_length=30, max_length=100):
        self.instructions = []
        self.length = random.randint(min_length, max_length)
        self.registers = [0] * num_registers  # Variable registers
        self.constants = [random.uniform(-1, 1) for _ in range(num_constants)]  # Constant registers
        self.operators = ['+', '-', '*', '/']  # You can extend this list
        self.initialize_instructions()
    
    def initialize_instructions(self):
        for _ in range(self.length//4):
            operator = random.choice(self.operators)
            dest_reg = random.randint(0, len(self.registers) - 1)
            operand1 = random.choice(self.registers + self.constants)
            operand2 = random.choice(self.registers + self.constants)
            self.instructions.append((operator, dest_reg, operand1, operand2))
    
    def evaluate(self, x, y, z):
        # Initialize the registers with inputs
        self.registers[0] = x
        self.registers[1] = y
        self.registers[2] = z
        
        # Execute each instruction
        for operator, dest_reg, op1, op2 in self.instructions:
            if operator == '+':
                self.registers[dest_reg] = op1 + op2
            elif operator == '-':
                self.registers[dest_reg] = op1 - op2
            elif operator == '*':
                self.registers[dest_reg] = op1 * op2
            elif operator == '/':
                if op2 != 0:  # Avoid division by zero
                    self.registers[dest_reg] = op1 / op2
        
        # The output joint angles are stored in registers [3, 4, 5] for theta1, theta2, theta3
        return self.registers[0], self.registers[1], self.registers[2]




In [4]:
# Create an initial population of chromosomes
def initialize_population(pop_size, num_registers, num_constants, chromosome_min, chromosome_max):
    population = []
    for _ in range(pop_size):
        population.append(Chromosome(num_registers, num_constants, chromosome_min, chromosome_max))
    return population

# Evaluate the fitness of the population
def evaluate_population(population, training_samples):
    fitness_scores = []
    for chrom in population:
        total_fitness = 0
        for sample in training_samples:
            x, y, z, theta1_true, theta2_true, theta3_true = sample
            theta1_pred, theta2_pred, theta3_pred = chrom.evaluate(x, y, z)
            fitness = fitness_function(x, y, z, theta1_pred, theta2_pred, theta3_pred)
            total_fitness += fitness
        fitness_scores.append(total_fitness / len(training_samples))
    return fitness_scores


def evolve_population(population, training_samples, generations, max_chrom_length, tournament_size, tournament_prob, mutation_rate):
     # Set up the progress bar for the number of generations
    progress_bar = tqdm(total=generations, desc="Evolving Generations", unit="gen")
    for generation in range(generations):
        fitness_scores = evaluate_population(population, training_samples)

        best_individual_index = fitness_scores.index.(max(fitness_scores))
        
        # Selection
        selected_population = tournament_selection(population, fitness_scores, tournament_size, tournament_prob)
        
        # Crossover
        new_population = []
        for i in range(0, len(selected_population), 2):
            parent1, parent2 = selected_population[i], selected_population[i+1]
            child1, child2 = two_point_crossover(parent1, parent2, max_chrom_length)
            new_population.extend([child1, child2])
        
        # Mutation
        new_population[0] = population[best_individual_index]
        for chrom in new_population:
            mutation(chrom, mutation_rate, chrom.operators)
        
        population = new_population
         # Update the progress bar after each generation
        progress_bar.set_postfix({"Best Fitness": max(fitness_scores)})  # Optional: show the best fitness in progress bar
        progress_bar.update(1)
     # Close the progress bar when evolution is done
    progress_bar.close()
    return population

In [5]:
def generate_training_data(num_samples):
    training_data = []
    for _ in range(num_samples):
        # Generate random joint angles within their respective ranges
        theta1 = random.uniform(0, 180)  # θ1 ∈ [0, 180]
        theta2 = random.uniform(0, 180)  # θ2 ∈ [0, 180]
        theta3 = random.uniform(0, 90)   # θ3 ∈ [0, 90]

        # Compute the corresponding end-effector position using forward kinematics
        x, y, z = forward_kinematics(theta1, theta2, theta3)
        
        # Append to training data (x, y, z, θ1, θ2, θ3)
        training_data.append((x, y, z, theta1, theta2, theta3))
    
    return training_data


In [6]:
# Step 1: Generate Training Data
training_samples = generate_training_data(num_samples=100)

# Step 2: Initialize Population
pop_size = 1000
min_chrom_length = 30
max_chrom_length = 100
tournament_size = 2
tournament_prob = 0.75
mutation_rate = 0.05
num_registers = 3
num_constants = 2
population = initialize_population(pop_size, num_registers, num_constants, min_chrom_length, max_chrom_length)

# Step 3: Evolve the Population
generations = 100
evolved_population = evolve_population(population, training_samples, generations, max_chrom_length, tournament_size, tournament_prob, mutation_rate)

# Step 4: Get the Best Chromosome
best_chromosome = min(evolved_population, key=lambda chrom: evaluate_population([chrom], training_samples)[0])

# Step 5: Test the Best Chromosome
test_point = (0.3, 0.15, 0.33)
theta1_pred, theta2_pred, theta3_pred = best_chromosome.evaluate(*test_point)

# Output results
#print("Best evolved chromosome instructions:")
#for instr in best_chromosome.instructions:
 #   print(instr)

print(f"Predicted joint angles for P = {test_point}:")
print(f"Theta1: {theta1_pred}, Theta2: {theta2_pred}, Theta3: {theta3_pred}")

end_effector = forward_kinematics(theta1_pred, theta2_pred, theta3_pred)
print(f"Predicted position; {end_effector}")

print(f"Error: {fitness_function(*test_point, theta1_pred, theta2_pred, theta3_pred)}")

Evolving Generations:   2%|▊                                      | 2/100 [00:32<27:21, 16.75s/gen, Best Fitness=0.286]
KeyboardInterrupt



In [ ]:
#print(f"Error: {fitness_function(*test_point, theta1_pred, theta2_pred, theta3_pred)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Link lengths (in meters)
L1, L2, L3, L4, L5, L6, L7, L8, L9 = 0.055, 0.315, 0.045, 0.108, 0.005, 0.034, 0.015, 0.088, 0.204

def forward_kinematics_plotting(theta1, theta2, theta3):
    """ Compute the 3D coordinates of each joint and the end-effector. """
    theta1, theta2, theta3 = np.radians([theta1, theta2, theta3])

    # Base
    x0, y0, z0 = 0, 0, L1
    base_position = np.array([x0, y0, z0, 1])
    
    # Transformation matrix from frame 0 to frame 1 (shoulder)
    T1 = np.array([
        [np.cos(theta1), -np.sin(theta1), 0, L6],
        [np.sin(theta1), np.cos(theta1), 0, -L4],
        [0, 0, 1, L2 + L3],
        [0, 0, 0, 1]
    ]) @ np.array([
        [1, 0, 0, 0],
        [0, 0, -1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1]
    ])
    shoulder_position = T1 @ base_position
    # Transformation matrix from frame 1 to frame 2 (elbow)
    T2 = np.array([
        [np.cos(theta2), -np.sin(theta2), 0, L7],
        [np.sin(theta2), np.cos(theta2), 0, -L8],
        [0, 0, 1, -L5],
        [0, 0, 0, 1]
    ])
       # Elbow position after applying T1 and T2
    elbow_position = T1 @ T2 @ base_position
    # Transformation matrix from frame 2 to frame 3 (hand)
    T3 = np.array([
        [np.cos(theta3), -np.sin(theta3), 0, 0],
        [np.sin(theta3), np.cos(theta3), 0, -L9],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    
    # Wrist (end-effector) position after applying T1, T2, and T3
    wrist_position = T1 @ T2 @ T3 @ base_position
    # Final transformation matrix from frame 0 to the end-effector
    T_final = T1 @ T2 @ T3
    
     # Extract joint positions (shoulder, elbow, wrist) for 3D plotting
    shoulder = shoulder_position[:3]  # x1, y1, z1
    elbow = elbow_position[:3]        # x2, y2, z2
    wrist = wrist_position[:3]        # x3, y3, z3
    
    return [x0, y0, z0], shoulder, elbow, wrist  # Return all joint positions

    #return np.array([[x0, y0, z0], [x1, y1, z1], [x2, y2, z2]])

def plot_robot_arm(joint_positions):
    """ Plot the 3D representation of the robot arm based on joint positions. """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Unpack joint positions
    base, shoulder, elbow, wrist = joint_positions

    # Extract x, y, z coordinates for each joint
    x_vals = [base[0], shoulder[0], elbow[0], wrist[0]]
    y_vals = [base[1], shoulder[1], elbow[1], wrist[1]]
    z_vals = [base[2], shoulder[2], elbow[2], wrist[2]]
    
    # Plot arm links between joints
    ax.plot(x_vals, y_vals, z_vals, marker='o', color='b', markersize=8, linewidth=2)
     # Plot wrist (end-effector) with red marker
    ax.scatter(wrist[0], wrist[1], wrist[2], color='r', s=100, label='Wrist (End-Effector)')

    # Set plot limits and labels
    ax.set_xlim([-0.2, 0.2])
    ax.set_ylim([-0.2, 0.2])
    ax.set_zlim([0, 0.5])
    ax.set_xlabel("X Axis")
    ax.set_ylabel("Y Axis")
    ax.set_zlabel("Z Axis")
    ax.set_title("3D Visualization of Hubert Robot Arm")

    plt.show()


# Example joint angles (in degrees)
theta1 = 0  # Base rotation
theta2 = 20  # Elbow angle
theta3 = 15  # Wrist angle



# Get the positions of all joints
#base, shoulder, elbow, wrist = forward_kinematics(theta1, theta2, theta3)


# Get the joint positions
joint_positions = forward_kinematics_plotting(theta1, theta2, theta3)

# Plot the robot arm in 3D
plot_robot_arm(joint_positions)
